In [1]:
import pandas as pd
data = pd.read_csv('preprocessed_data_with_numerical.csv')

In [2]:
data.fillna('', inplace=True)

In [3]:
data['full_text'] = data['title'] + " " + data['location']  + " " + data['department']  + " " + data['company_profile']  + " " + data['description']  + " " + data['requirements']  + " "  + data['benefits'] + data['industry']  + " " + data['function']

In [4]:
data

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset,missing company profile,missing company information,full_text
0,marketing intern,usa ny new york,marketing,food52 created groundbreaking award winning co...,food52 fast growing james beard award winning ...,experience content management system major plu...,,0,1,0,2,4,9,,marketing,0,0,1,3,marketing intern usa ny new york marketing foo...
1,customer service cloud video production,nz auckland,success,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get u part 90 second team gain experience work...,0,1,0,1,6,9,marketing advertising,customer service,0,0,1,3,customer service cloud video production nz auc...
2,commissioning machinery assistant cma,usa ia wever,,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre commissioning commissioning proc...,,0,1,0,2,6,9,,,0,0,1,3,commissioning machinery assistant cma usa ia w...
3,account executive washington dc,usa dc washington,sale,passion improving quality life geography heart...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,1,5,1,computer software,sale,0,0,1,3,account executive washington dc usa dc washing...
4,bill review manager,usa fl fort worth,,spotsource solution llc global human capital m...,job title itemization review manager location ...,qualification rn license state texas diploma b...,full benefit offered,0,1,1,1,5,1,hospital health care,health care provider,0,0,1,3,bill review manager usa fl fort worth spotsou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,account director distribution,ca toronto,sale,vend looking awesome new talent come join u wo...,case first time visited website vend award win...,ace role eat comprehensive statement work brea...,expect u open culture openly share result inpu...,0,1,1,1,5,9,computer software,sale,0,0,1,3,account director distribution ca toronto sale ...
17876,payroll accountant,usa pa philadelphia,accounting,weblinc e commerce platform service provider f...,payroll accountant focus primarily payroll fun...,b b accounting desire fun love genuine passion...,health wellness medical plan prescription drug...,0,1,1,1,5,1,internet,accounting auditing,0,0,1,3,payroll accountant usa pa philadelphia account...
17877,project cost control staff engineer cost contr...,usa tx houston,,provide full time permanent position many medi...,experienced project cost control staff enginee...,least 12 year professional experience ability ...,,0,0,0,1,6,9,,,0,0,1,2,project cost control staff engineer cost contr...
17878,graphic designer,ng la lagos,,,nemsia studio looking experienced visual graph...,1 must fluent latest version corel adobe cc es...,competitive salary compensation based experien...,0,0,1,0,6,6,graphic design,design,0,0,0,0,graphic designer ng la lagos nemsia studio l...


In [5]:
X = data[['full_text', 'has_questions', 'required_experience', 'required_education', 'missing company information']]
# X = data['full_text']
y = data['fraudulent']
from sklearn.model_selection import train_test_split
X_train, X_test,y_train , y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
# from sklearn.pipeline import FeatureUnion, Pipeline
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

<h1>Text Only Models</h1>

TFIDF Vectorizer

In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['full_text'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['full_text'])

In [8]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_tfidf, y_train, X_test_tfidf, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

Logistic Regression F1 Score: 0.625
SVM F1 Score: 0.8153846153846154
Random Forest F1 Score: 0.793733681462141
XGBoost F1 Score: 0.8226600985221675


Word2Vec

In [9]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# Step 2: Tokenize the text
tokenized_text = X_train['full_text'].apply(lambda x: x.split())

# Step 3: Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Step 4: Convert words to vectors
def get_vector(word_list, model):
    valid_words = [word for word in word_list if word in model.wv]
    if not valid_words:
        # If no valid words, return a vector of zeros or handle as needed
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

X_train_word2vec = tokenized_text.apply(lambda x: get_vector(x, word2vec_model))
X_test_word2vec = X_test['full_text'].apply(lambda x: get_vector(x.split(), word2vec_model))

In [10]:
# Step 5: Train models and evaluate F1 score
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train.to_list(), y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test.to_list())
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 6: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 7: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 8: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_word2vec, y_train, X_test_word2vec, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/natthaphonkanthawang/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.36526946107784425
SVM F1 Score: 0.4875
Random Forest F1 Score: 0.5818181818181818
XGBoost F1 Score: 0.7229551451187335


CountVectorizer

In [11]:
count_vectorizer = CountVectorizer()
X_train_cv = count_vectorizer.fit_transform(X_train['full_text'])
X_test_cv = count_vectorizer.transform(X_test['full_text'])

In [12]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Step 4: Logistic Regression
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, X_train_cv, y_train, X_test_cv, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/natthaphonkanthawang/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.8009153318077803
SVM F1 Score: 0.7272727272727272
Random Forest F1 Score: 0.793733681462141
XGBoost F1 Score: 0.8382352941176471


<h3>Using text features only</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost | Average |
|----------|----------|----------|----------|----------|----------|
| TFIDF | 0.625 | 0.815 | 0.787 | 0.827 | 0.764 |
| Word2Vec | 0.401 | 0.509 | 0.571 | 0.699 | 0.545 |
| CountVectorizer | 0.801 | 0.727 | 0.794 | 0.838 | 0.790 |


Based on the results above, the best word embedding method to use is CountVectorizer that obtained the highest average F1 score of 0.790 across all models. Hence, we will be using CountVectorizer moving forward.

 Long Short Term Memory (LSTM)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot

2023-11-22 22:26:48.016426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
voc_size=10000
corpus = data["full_text"]
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]


[8802,
 4572,
 9945,
 6877,
 8100,
 2744,
 3904,
 1903,
 4295,
 9703,
 8122,
 9945,
 6877,
 8100,
 4572,
 4295,
 9703,
 8122,
 9945,
 6877,
 8100,
 4572,
 325,
 8829,
 8947,
 5708,
 3470,
 7766,
 3820,
 6877,
 2060,
 4570,
 6298,
 8371,
 8122,
 4295,
 9703,
 8358,
 6877,
 8100,
 4024,
 3964,
 2961,
 6184,
 9945,
 2620,
 1430,
 6202,
 4295,
 3846,
 5564,
 4295,
 9703,
 7636,
 931,
 1889,
 6353,
 9730,
 9833,
 1467,
 9779,
 6877,
 8100,
 4745,
 9426,
 7144,
 5817,
 6877,
 746,
 4220,
 3820,
 387,
 5964,
 9291,
 3101,
 7284,
 5056,
 95,
 6877,
 5953,
 3172,
 9333,
 2961,
 1954,
 8100,
 1903,
 5330,
 3000,
 9333,
 4295,
 9703,
 5006,
 9475,
 1903,
 6001,
 4295,
 9703,
 6298,
 1598,
 2530,
 5056,
 6877,
 5412,
 9333,
 8769,
 9291,
 8829,
 8563,
 8122,
 2032,
 8563,
 4650,
 1370,
 7761,
 5683,
 6720,
 6548,
 3904,
 1258,
 5742,
 6925,
 718,
 6202,
 4295,
 3846,
 5564,
 6202,
 4295,
 3846,
 9898,
 6202,
 4295,
 3846,
 9582,
 8979,
 9892,
 4267,
 2378,
 2703,
 8802,
 4572,
 7946,
 4411,
 3607,

In [15]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[9592 7882 2268 ... 7431 7963 2792]
 [8111 6654 6547 ...  864 8802 4572]
 [5316 2934 4284 ... 2477  746 3017]
 ...
 [4274 8404 3717 ... 7678 3846 7354]
 [3911 5459 6350 ... 5523 5459 5459]
 [7525 9118 3617 ... 3140 9436 5010]]


In [16]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2023-11-22 22:26:57.361952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            500000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(embedded_docs, y, test_size=0.3, random_state=0)

In [18]:
model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))

Epoch 1/10
392/392 [==============================] - 45s 103ms/step - loss: 0.1480 - accuracy: 0.9618 - val_loss: 0.0816 - val_accuracy: 0.9782
Epoch 2/10
392/392 [==============================] - 35s 90ms/step - loss: 0.0576 - accuracy: 0.9839 - val_loss: 0.0732 - val_accuracy: 0.9799
Epoch 3/10
392/392 [==============================] - 35s 89ms/step - loss: 0.0241 - accuracy: 0.9926 - val_loss: 0.0821 - val_accuracy: 0.9752
Epoch 4/10
392/392 [==============================] - 42s 107ms/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.1032 - val_accuracy: 0.9802
Epoch 5/10
392/392 [==============================] - 40s 101ms/step - loss: 0.0034 - accuracy: 0.9990 - val_loss: 0.1149 - val_accuracy: 0.9808
Epoch 6/10
392/392 [==============================] - 36s 92ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.1392 - val_accuracy: 0.9795
Epoch 7/10
392/392 [==============================] - 36s 92ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 0.1260 - val_accura

In [19]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_lstm, y_test_lstm)

print(f"Test Accuracy: {accuracy}")
print(f"Test Loss: {loss}")

168/168 [==============================] - 5s 29ms/step - loss: 0.1385 - accuracy: 0.9661
Test Accuracy: 0.9660701155662537
Test Loss: 0.13849139213562012


In [20]:
y_pred = model.predict(X_test_lstm)

168/168 [==============================] - 6s 32ms/step


In [21]:
from sklearn.metrics import classification_report
y_pred_binary = (y_pred > 0.5).astype('int32')  
# Get classification report
report = classification_report(y_test_lstm, y_pred.round(),target_names = ['0','1'])
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      5137
           1       0.59      0.67      0.62       227

    accuracy                           0.97      5364
   macro avg       0.79      0.82      0.80      5364
weighted avg       0.97      0.97      0.97      5364



In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_binary)

array([[5031,  106],
       [  76,  151]])

In [23]:
from sklearn.metrics import f1_score
f1_test = f1_score(y_test_lstm, y_pred_binary )
print(f'LSTM F1 score: {f1_test}')

LSTM F1 score: 0.6239669421487604


<h1>Combined text and numeric</h1>

tfidf and numeric

In [24]:
### TO CHANGE, ABIT LAZY REMOVE 22 NOV 22:30
X_train, X_test,y_train , y_test = train_test_split(data.drop('fraudulent', axis=1), data["fraudulent"], test_size=0.3, random_state=0)

In [25]:
numeric_features = X_train[['has_questions', 'required_experience', 'required_education', 'missing company information']]
combined_features = np.hstack([
    StandardScaler().fit_transform(numeric_features),
    X_train_cv.toarray()])
X_test_numeric_features = X_test[['has_questions', 'required_experience', 'required_education', 'missing company information']]
X_test_combined_features = np.hstack([
    StandardScaler().fit_transform(X_test_numeric_features),
    X_test_cv.toarray()])

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate F1 score
    f1 = f1_score(y_test, y_pred)
    
    return f1
logreg_model = LogisticRegression()
logreg_f1 = train_and_evaluate_model(logreg_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Logistic Regression F1 Score: {logreg_f1}")

# Step 5: Support Vector Machine (SVM)
svm_model = SVC()
svm_f1 = train_and_evaluate_model(svm_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"SVM F1 Score: {svm_f1}")

# Step 6: Random Forest
rf_model = RandomForestClassifier()
rf_f1 = train_and_evaluate_model(rf_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"Random Forest F1 Score: {rf_f1}")

xgb_model = XGBClassifier()
xgb_f1 = train_and_evaluate_model(xgb_model, combined_features, y_train, X_test_combined_features, y_test)
print(f"XGBoost F1 Score: {xgb_f1}")

/Users/natthaphonkanthawang/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression F1 Score: 0.8232662192393735
SVM F1 Score: 0.7287671232876712
Random Forest F1 Score: 0.793733681462141
XGBoost F1 Score: 0.8463356973995273


<h3>Combined features</h3>

|  | Logistic Regression | SVM | Random Forest | XGBoost |
|----------|----------|----------|----------|----------|
| Text Only | 0.801 | 0.727 | 0.794 | 0.838 |
| Text and Numeric | 0.823 | 0.729 | 0.791 | 0.846 |